In [ ]:
import time
import asyncio
import tracemalloc
from loguru import logger
from tqdm import tqdm, trange
from random import randint, choice
from proxy_client import ProxyProvider
from requests_html import AsyncHTMLSession

tracemalloc.start()

def setup_logger(file_name):
    logger.remove()
    logger.add(f"./logs/{file_name}-debug.log", format="{time} {level} {message}", level="DEBUG", enqueue=True)
    logger.add(f"./logs/{file_name}-info.log", format="{time} {level} {message}", level="INFO", enqueue=True, backtrace=True)
    logger.add(f"./logs/{file_name}-error.log", format="{time} {level} {message}", level="ERROR", enqueue=True, backtrace=True, diagnose=True)

setup_logger("proxy-refine")


proxy_provider = ProxyProvider()
asession = AsyncHTMLSession()
urls = [
    "https://divar.ir/s/tehran/jobs",
    "https://instagram.com", 
    "https://www.googleapis.com/discovery/v1/apis/drive/v3/rest"
    "https://www.tse.ir",
    "http://tsetmc.ir",
    "http://tsetmc.ir/Loader.aspx?ParTree=15",
    "http://www.fipiran.com",
#     "https://core.telegram.org/bots",
#     "http://api.telegram.org"
    
]
# urls = ["https://www.googleapis.com/discovery/v1/apis/drive/v3/rest", "https://balad.ir", 
#         "https://google.com", "https://youtube.com", "https://facebook.com", "https://baidu.com",
#         "https://wikipedia.org", "https://twitter.com", "https://yahoo.com", "https://instagram.com",
#         "https://amazon.com"]
async def getit(proxy_provider):
    i = 0
    while i < 100000:
        logger.debug(f"starting {i}")
        i += 1
        try:
            url = choice(urls)
            
            url_protocol = url.split("://")[0]
            rand = randint(0, 1)
            protocol = url_protocol #"socks5"
            if url_protocol == "http" and rand == 1:
                protocol = "socks5"
            if i % 5 == 0:
                p = proxy_provider.get_proxy(protocol)
            else:
                p = proxy_provider.get_bad_proxy(protocol)
            if p == None or p == "None":
                time.sleep(10)
                continue
            r = await asession.get(f"{url_protocol}://google.com", proxies={protocol: p}, timeout=15)
            if "google.com" in r.text:
                r = await asession.get(url, proxies={protocol: p}, timeout=15)
                if r.status_code != 200:
                    proxy_provider.bad_ip(protocol, p)
                else:
                    print(protocol, p)
            else:
                proxy_provider.bad_ip(protocol, p)
            del p
            del r
        except:
            proxy_provider.bad_ip(protocol, p)
            del p

tasks = []

for i in trange(20):
    tasks.append(getit(proxy_provider))
results = await asyncio.wait(tasks)

100%|██████████| 20/20 [00:00<00:00, 12961.38it/s]


socks5 117.135.198.33:80
socks5 109.74.32.10:8080
socks5 211.33.143.217:8080
socks5 182.52.137.135:8080
socks5 41.60.236.234:6833
socks5 187.37.244.25:40966
socks5 180.245.64.46:8080
socks5 187.122.112.126:3128
socks5 221.206.5.51:8080
socks5 192.162.62.197:59246
socks5 52.32.29.138:8083
socks5 5.160.115.126:8080
socks5 122.143.85.223:38801
socks5 206.189.140.91:80
socks5 104.196.249.140:80
socks5 138.122.202.210:3128
socks5 188.72.9.82:51327
socks5 203.67.162.82:8088
socks5 5.9.87.163:80
socks5 218.92.227.165:18888
http 58.220.95.11:9400
http 170.82.52.48:8080
socks5 186.27.120.38:80
socks5 45.76.47.24:1189
socks5 115.236.37.18:808
socks5 122.5.135.17:808
socks5 89.40.112.22:1189
socks5 201.242.78.71:8080
socks5 194.87.234.42:3128
socks5 180.245.255.175:8080
socks5 110.5.98.162:80
socks5 123.179.29.102:8080
socks5 203.73.50.30:8088
http 58.220.95.39:10046
socks5 188.240.60.7:32473
socks5 191.102.125.243:8080
socks5 138.36.141.34:64101
socks5 138.121.91.159:8080
http 51.158.172.165:881

0

In [ ]:
import requests
import re
from time import gmtime, strftime, sleep
from tqdm import trange, tqdm
import time
import asyncio
import tracemalloc
from loguru import logger
from tqdm import tqdm, trange
from random import randint, choice
from proxy_client import ProxyProvider
from requests_html import AsyncHTMLSession
tracemalloc.start()

urls = []
with open("proxy_url.txt", "r") as f:
    for line in f.readlines():
        urls.append(line[:-1])

def setup_logger(file_name):
    logger.remove()
    logger.add(f"./logs/{file_name}-debug.log", format="{time} {level} {message}", level="DEBUG", enqueue=True)
    logger.add(f"./logs/{file_name}-info.log", format="{time} {level} {message}", level="INFO", enqueue=True, backtrace=True)
    logger.add(f"./logs/{file_name}-error.log", format="{time} {level} {message}", level="ERROR", enqueue=True, backtrace=True, diagnose=True)

setup_logger("proxy")


asession = AsyncHTMLSession()
q_url = asyncio.Queue()
q_goods = asyncio.Queue()
q_proxies = asyncio.Queue()
for url in tqdm(urls):
    await q_url.put(url)


async def getit(q_url, q_goods, q_proxies):
    while not q_url.empty():
        url = await q_url.get()
        logger.debug(f"start {url}")
        try:
            resp = await asession.get(url, timeout=15)
            regs = re.findall(r'((?:\d{1,3}\.){3}\d{1,3}):(\d+)', resp.text)
            for reg in regs:
                await q_proxies.put(reg[0] + ":" + reg[1])
            await q_goods.put(url)
            logger.info(f"done url {url}")
        except:
            logger.debug(f"bad url {url}")
        q_url.task_done()
        

tasks = []
for i in trange(50):
    logger.info("start runner")
    tasks.append(getit(q_url, q_goods, q_proxies))
results = await asyncio.wait(tasks)

proxies = []
while not q_proxies.empty():
    p = await q_proxies.get()
    proxies.append(p)
    
goods = []
while not q_goods.empty():
    g = await q_goods.get()
    goods.append(g)

100%|██████████| 50/50 [00:00<00:00, 1794.98it/s]
